In [1]:
import tifffile
import os
from natsort import os_sorted
import numpy as np
import napari
import json
from skimage import filters
#viewer = napari.Viewer()

# For single tile images

In [2]:
# ask for folder name with tkinter
from tkinter import Tk
from tkinter.filedialog import askdirectory
Tk().withdraw()
home = askdirectory(title='Select the folder with the tiff files') + '/'
files = os_sorted(os.listdir(home))
stack =  np.array([tifffile.imread(home + file) for file in files if file.endswith('.tif')])


In [3]:
metadata_file = home + 'metadata.json'
metadata = json.load(open(metadata_file))
xy_pixel_size = metadata['detector_pixel_size']
z_spacing = metadata['sample_stage_step_size']


In [ ]:
# uneven illumination correction
normalisation = '//storage3.ad.scilifelab.se/alm/BrismarGroup/DescSPIM/Data/241112_normalisation/13h48m02s_snap_thorlabscam.tiff'
norm = tifffile.imread(normalisation)


In [4]:
def uneven_illumination_correction(image,
                                   norm = '//storage3.ad.scilifelab.se/alm/BrismarGroup/DescSPIM/Data/241112_normalisation/13h48m02s_snap_thorlabscam.tiff', 
                                   sigma=100):
    norm_array = tifffile.imread(norm)
    norm_gauss = filters.gaussian(norm_array, sigma)
    return image/norm_gauss


def reshape_tiff_data(data, num_channels, normalise=False):
    # Initialize the output array with the desired shape (3, 250, 500, 500)
    zsteps = len(data)//num_channels
    output = np.zeros((num_channels, zsteps, data.shape[1], data.shape[2]), dtype=np.uint16)

    # Iterate over the z-steps
    for z in range(len(data)//num_channels):
        if z % 2 == 0:
            for c in range(num_channels):
                output[c, z] = data[z * num_channels + c] if not normalise else uneven_illumination_correction(data[z * num_channels + c])
        else:
            for c in range(num_channels):
                ci = num_channels - c - 1
                output[ci, z] = data[z * num_channels + c] if not normalise else uneven_illumination_correction(data[z * num_channels + c])
    return output

def reshape_tiff_data(data, num_channels):
    # Initialize the output array with the desired shape (3, 250, 500, 500)
    zsteps = len(data)//num_channels
    output = np.zeros((num_channels, zsteps, data.shape[1], data.shape[2]), dtype=np.uint16)

    # Iterate over the z-steps
    for z in range(len(data)//num_channels):
        if z % 2 == 0:
            for c in range(num_channels):
                output[c, z] = data[z * num_channels + c]
        else:
            for c in range(num_channels):
                ci = num_channels - c - 1
                output[ci, z] = data[z * num_channels + c]
    return output

# Example usage
reshaped_data = reshape_tiff_data(stack, 2)


In [5]:
save_path = home[:-1] + '_reshaped_data/'

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
for c in range(reshaped_data.shape[0]):
    ##viewer.add_image(reshaped_data[c], name=str(c), colormap='gray', scale=(1.7097, 0.345, 0.345), blending='additive')
    tifffile.imwrite(save_path + home.split('/')[-2] + '_channel_' + str(c) + '.tif', 
                     reshaped_data[c].astype(np.uint16),
                     resolution=(1/xy_pixel_size, 1/xy_pixel_size),
                     imagej=True,
                     metadata={'spacing': z_spacing,'unit':'um', 'axes':'ZYX', 
                               'PhysicalSizeY':xy_pixel_size, 'PhysicalSizeX':xy_pixel_size,
                               'PhysicalSizeYUnit': 'um','PhysicalSizeXUnit': 'um'},)

c:\Users\alm\anaconda3\envs\ims_des\lib\site-packages\tifffile\tifffile.py:3786: UserWarning: <tifffile.TiffWriter '14h43m32s_rec_t…am_channel_0.tif'> truncating ImageJ file
  warnings.warn(
c:\Users\alm\anaconda3\envs\ims_des\lib\site-packages\tifffile\tifffile.py:3786: UserWarning: <tifffile.TiffWriter '14h43m32s_rec_t…am_channel_1.tif'> truncating ImageJ file
  warnings.warn(


In [5]:
#Steve made this because metadata was not saved correctly
save_path = home[:-1] + '_reshaped_data/'

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
for c in range(reshaped_data.shape[0]):
    ##viewer.add_image(reshaped_data[c], name=str(c), colormap='gray', scale=(1.7097, 0.345, 0.345), blending='additive')
    tifffile.imwrite(save_path + home.split('/')[-2] + '_channel_' + str(c) + '.tif', 
                     reshaped_data[c].astype(np.uint16),
                     imagej=True,)

c:\Users\alm\anaconda3\envs\ims_des\lib\site-packages\tifffile\tifffile.py:3786: UserWarning: <tifffile.TiffWriter 'kidney_tile_5_s…am_channel_0.tif'> truncating ImageJ file
  warnings.warn(
c:\Users\alm\anaconda3\envs\ims_des\lib\site-packages\tifffile\tifffile.py:3786: UserWarning: <tifffile.TiffWriter 'kidney_tile_5_s…am_channel_1.tif'> truncating ImageJ file
  warnings.warn(


# For multi tile images

In [12]:
def reshape_tiff_data(data, num_channels):
    # Initialize the output array with the desired shape (3, 250, 500, 500)
    zsteps = len(data)//num_channels
    output = np.zeros((num_channels, zsteps, data.shape[1], data.shape[2]), dtype=np.uint16)

    # Iterate over the z-steps
    for z in range(len(data)//num_channels):
        if z % 2 == 0:
            for c in range(num_channels):
                output[c, z] = data[z * num_channels + c]
        else:
            for c in range(num_channels):
                ci = num_channels - c - 1
                output[ci, z] = data[z * num_channels + c]

    return output

In [13]:
home = 'C:/Users/alm/Documents/ImSwitchConfig/recordings/2024-09-02/closed_slit_rec_thorlabscam/'
save_path = 'C:/Users/alm/Documents/ImSwitchConfig/recordings/2024-09-02/closed_slit_rec_thorlabscam_reshaped/'

metadata_file = home + 'metadata.json'
metadata = json.load(open(metadata_file))
xy_pixel_size = metadata['detector_pixel_size']
z_spacing = metadata['sample_stage_step_size']

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
tiles = os_sorted(os.listdir(home))
outs = os_sorted(os.listdir(save_path))

for i, tile in enumerate(tiles):
    if f'tile_{i}_channel_0.tif' in outs:
        print(f'{tile} already processed, skipping...')
    else:
        files = os_sorted(os.listdir(home + tile + '/'))        #home + tile + '/'
        stack =  np.array([tifffile.imread(home + tile + '/' + file) for file in files])
        print(f'processing {tile}, stack shape: {stack.shape}')

        reshaped_data = reshape_tiff_data(stack, 2)
        for c in range(reshaped_data.shape[0]):
            print(f'saving {save_path}tile_{i}_channel_{c}.tif ...') 
            tifffile.imwrite(f'{save_path}tile_{i}_channel_{c}.tif',
                            reshaped_data[c,:,:,:],
                            imagej=True,
                            metadata={'spacing': z_spacing,'unit':'um', 'axes':'ZYX', 
                                    'PhysicalSizeY':xy_pixel_size, 'PhysicalSizeX':xy_pixel_size,
                                    'PhysicalSizeYUnit': 'um','PhysicalSizeXUnit': 'um'},
                            resolution=(1/xy_pixel_size, 1/xy_pixel_size)

                            )

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/alm/Documents/ImSwitchConfig/recordings/2024-09-02/closed_slit_rec_thorlabscam/image_0.tif/'

In [5]:
#viewer.add_image(stack, name='stack')#, colormap='green', blending='additive')

